In [1]:
import jenkspy
import pandas as pd
from utils import PostGresLoad

%load_ext autoreload
%autoreload 2

In [2]:
pg = PostGresLoad()

In [3]:
df_11_12 = pg.df_from_rds("""select * from dissimilarity_index where "YEAR" = '2011-12';""")
df_13_14 = pg.df_from_rds("""select * from dissimilarity_index where "YEAR" = '2013-14';""")
df_15_16 = pg.df_from_rds("""select * from dissimilarity_index where "YEAR" = '2015-16';""")
df_17_18 = pg.df_from_rds("""select * from dissimilarity_index where "YEAR" = '2017-18';""")

In [7]:
def classify(x, l1, l2):
    if x is None:
        return x
    if x <= l1:
        return 'low'
    if x <= l2:
        return 'med'
    if x < 1.0:
        return 'high'

In [8]:
def categorize_dscore(df_column):
    list_of_values = df_column
    breaks = jenkspy.jenks_breaks(list_of_values, nb_class=6)
    lim1 = breaks[2]
    lim2 = breaks[4]
    return df_column.apply(lambda x: classify(x, lim1, lim2))

In [9]:
frames = [df_11_12, df_13_14, df_15_16, df_17_18]

In [10]:
for df in frames:
    for col in df.columns:
        if 'd_' in col:
            df[f'classified_{col}'] = categorize_dscore(df[col])


/Users/cave/miniconda3/lib/python3.7/site-packages/jenkspy/core.py:64: UserWarning: Invalid values encountered (NaN or Inf) were ignored
  warnings.warn('Invalid values encountered (NaN or Inf) were ignored')


In [17]:
df_17_18.shape

(2934, 14)

In [18]:
print(df_15_16.shape)

(2455, 14)


In [19]:
print(df_13_14.shape)

(2448, 14)


In [20]:
df_final = pd.concat([df_11_12, df_13_14, df_15_16, df_17_18])
df_final.to_csv('/Users/cave/Desktop/pleiades_archive/discriminology/output/classified_scores.csv')

In [16]:
df_final.columns

Index(['LEAID', 'YEAR', 'd_wh_bl', 'd_wh_as', 'd_wh_am', 'd_wh_tr', 'd_wh_hp',
       'd_wh_hi', 'classified_d_wh_bl', 'classified_d_wh_as',
       'classified_d_wh_am', 'classified_d_wh_tr', 'classified_d_wh_hp',
       'classified_d_wh_hi'],
      dtype='object')